# Chapter 36: Building an NFT Marketplace

---

Non-fungible tokens (NFTs) have revolutionized digital ownership, enabling unique assets like art, collectibles, and in-game items. An NFT marketplace is a platform where users can list, buy, and sell these tokens. In this chapter, we'll build a complete NFT marketplace from scratch. Our marketplace will support listing NFTs for sale at a fixed price, purchasing them, and optionally conducting auctions. We'll also implement royalty payments for creators, integrate IPFS for metadata storage, and build a frontend to interact with the contracts. By the end, you'll have a functional NFT marketplace and a deep understanding of the mechanics behind platforms like OpenSea and Rarible.

---

## 36.1 Project Overview

### 36.1.1 Requirements and Features

Our NFT marketplace will include the following features:

- **Listing NFTs**: Any user can list their ERC-721 token for sale at a fixed price (in ETH).
- **Buying NFTs**: Users can purchase listed NFTs by paying the asking price, which is transferred to the seller.
- **Canceling listings**: Sellers can cancel their listings before a purchase occurs.
- **Auctions** (optional but included): Sellers can create English auctions with a minimum bid, bid increment, and duration. The highest bidder wins and pays their bid.
- **Royalties**: Creators can earn a percentage (e.g., 5%) on secondary sales, enforced by the marketplace.
- **Marketplace fees**: The platform can take a small fee (e.g., 2.5%) from each sale.
- **Metadata display**: NFTs have associated metadata (name, description, image) stored on IPFS.

### 36.1.2 Architecture Design

We'll need several smart contracts:

- **NFT Contract**: A basic ERC-721 contract (or we can use any existing NFT).
- **Marketplace Contract**: The core contract handling listings, purchases, auctions, and fees.
- **Royalty Management**: Logic to track and pay royalties to creators.

Optionally, we could separate auction logic into a separate contract, but for simplicity, we'll include it in the marketplace.

```
┌─────────────────────────────────────────────────────────────┐
│                      Marketplace                             │
│  • listings (mapping: listingId → Listing)                  │
│  • auctions (mapping: auctionId → Auction)                  │
│  • listItem()                                               │
│  • buyItem()                                                │
│  • cancelListing()                                          │
│  • createAuction()                                          │
│  • placeBid()                                               │
│  • endAuction()                                             │
└─────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│                         NFT Contract                         │
│  • ERC-721 token                                            │
│  • mint() (only owner)                                      │
│  • transfer/safeTransfer                                    │
└─────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│                         IPFS                                 │
│  • Store metadata (JSON + image)                            │
│  • tokenURI points to IPFS hash                             │
└─────────────────────────────────────────────────────────────┘
```

**Data Structures:**

- **Listing**: `id`, `tokenAddress`, `tokenId`, `seller`, `price`, `active`
- **Auction**: `id`, `tokenAddress`, `tokenId`, `seller`, `startTime`, `endTime`, `minBid`, `bidIncrement`, `highestBidder`, `highestBid`, `ended`

---

## 36.2 Smart Contract Development

### 36.2.1 NFT Contract

We'll use a simple ERC-721 contract with minting capabilities (only owner can mint for simplicity). In practice, you might use OpenZeppelin's ERC721 and add your own logic.

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

import "@openzeppelin/contracts/token/ERC721/ERC721.sol";
import "@openzeppelin/contracts/access/Ownable.sol";
import "@openzeppelin/contracts/utils/Counters.sol";

contract SimpleNFT is ERC721, Ownable {
    using Counters for Counters.Counter;
    Counters.Counter private _tokenIds;

    // Mapping from tokenId to creator address (for royalties)
    mapping(uint256 => address) private _creators;

    event NFTMinted(address indexed creator, uint256 indexed tokenId, string tokenURI);

    constructor() ERC721("SimpleNFT", "SNFT") {}

    function mintNFT(address recipient, string memory tokenURI) public onlyOwner returns (uint256) {
        _tokenIds.increment();
        uint256 newItemId = _tokenIds.current();
        _mint(recipient, newItemId);
        _setTokenURI(newItemId, tokenURI);
        _creators[newItemId] = recipient;
        emit NFTMinted(recipient, newItemId, tokenURI);
        return newItemId;
    }

    function creatorOf(uint256 tokenId) public view returns (address) {
        return _creators[tokenId];
    }

    // Optional: Override to support royalties (EIP-2981)
    function royaltyInfo(uint256 tokenId, uint256 salePrice) external view returns (address receiver, uint256 royaltyAmount) {
        receiver = _creators[tokenId];
        royaltyAmount = (salePrice * 5) / 100; // 5% royalty
    }
}
```

### 36.2.2 Marketplace Contract

Now the main marketplace contract. We'll implement fixed-price listings and auctions.

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

import "@openzeppelin/contracts/token/ERC721/IERC721.sol";
import "@openzeppelin/contracts/security/ReentrancyGuard.sol";
import "@openzeppelin/contracts/access/Ownable.sol";
import "@openzeppelin/contracts/utils/Counters.sol";

contract NFTMarketplace is ReentrancyGuard, Ownable {
    using Counters for Counters.Counter;

    // --- Structs ---
    struct Listing {
        uint256 id;
        address tokenAddress;
        uint256 tokenId;
        address payable seller;
        uint256 price;
        bool active;
    }

    struct Auction {
        uint256 id;
        address tokenAddress;
        uint256 tokenId;
        address payable seller;
        uint256 startTime;
        uint256 endTime;
        uint256 minBid;
        uint256 bidIncrement;
        address payable highestBidder;
        uint256 highestBid;
        bool ended;
    }

    // --- State Variables ---
    Counters.Counter private _listingIds;
    Counters.Counter private _auctionIds;

    mapping(uint256 => Listing) public listings;
    mapping(uint256 => Auction) public auctions;

    // Platform fee in basis points (e.g., 250 = 2.5%)
    uint256 public platformFeeBasisPoints = 250;
    address payable public feeRecipient;

    // Royalty basis points (default 5% = 500)
    uint256 public constant ROYALTY_BASIS_POINTS = 500;

    // --- Events ---
    event ListingCreated(
        uint256 indexed id,
        address indexed tokenAddress,
        uint256 indexed tokenId,
        address seller,
        uint256 price
    );
    event ListingPurchased(
        uint256 indexed id,
        address indexed buyer,
        uint256 price
    );
    event ListingCanceled(uint256 indexed id);

    event AuctionCreated(
        uint256 indexed id,
        address indexed tokenAddress,
        uint256 indexed tokenId,
        address seller,
        uint256 startTime,
        uint256 endTime,
        uint256 minBid
    );
    event BidPlaced(uint256 indexed id, address indexed bidder, uint256 amount);
    event AuctionEnded(uint256 indexed id, address winner, uint256 amount);

    // --- Modifiers ---
    modifier listingExists(uint256 listingId) {
        require(listings[listingId].active, "Listing does not exist or is inactive");
        _;
    }

    modifier auctionExists(uint256 auctionId) {
        require(auctions[auctionId].tokenAddress != address(0), "Auction does not exist");
        _;
    }

    modifier auctionActive(uint256 auctionId) {
        Auction storage auction = auctions[auctionId];
        require(block.timestamp >= auction.startTime && block.timestamp < auction.endTime, "Auction not active");
        require(!auction.ended, "Auction already ended");
        _;
    }

    constructor(address payable _feeRecipient) {
        feeRecipient = _feeRecipient;
    }

    // --- Listing Functions ---

    /**
     * @dev List an NFT for sale at a fixed price.
     * @param tokenAddress The address of the NFT contract.
     * @param tokenId The ID of the NFT.
     * @param price The sale price in wei.
     */
    function listItem(
        address tokenAddress,
        uint256 tokenId,
        uint256 price
    ) external nonReentrant {
        require(price > 0, "Price must be greater than zero");
        IERC721 nft = IERC721(tokenAddress);
        require(nft.ownerOf(tokenId) == msg.sender, "You do not own this token");
        require(nft.isApprovedForAll(msg.sender, address(this)) || nft.getApproved(tokenId) == address(this),
            "Marketplace not approved to transfer token");

        _listingIds.increment();
        uint256 listingId = _listingIds.current();

        listings[listingId] = Listing({
            id: listingId,
            tokenAddress: tokenAddress,
            tokenId: tokenId,
            seller: payable(msg.sender),
            price: price,
            active: true
        });

        emit ListingCreated(listingId, tokenAddress, tokenId, msg.sender, price);
    }

    /**
     * @dev Buy a listed NFT.
     * @param listingId The ID of the listing.
     */
    function buyItem(uint256 listingId) external payable nonReentrant listingExists(listingId) {
        Listing storage listing = listings[listingId];
        require(msg.value >= listing.price, "Insufficient payment");

        // Transfer payment (after deducting fees and royalties)
        _distributePayment(listing.tokenAddress, listing.tokenId, listing.seller, listing.price);

        // Transfer NFT
        IERC721(listing.tokenAddress).safeTransferFrom(listing.seller, msg.sender, listing.tokenId);

        // Deactivate listing
        listing.active = false;

        // Refund excess payment
        if (msg.value > listing.price) {
            payable(msg.sender).transfer(msg.value - listing.price);
        }

        emit ListingPurchased(listingId, msg.sender, listing.price);
    }

    /**
     * @dev Cancel a listing.
     * @param listingId The ID of the listing.
     */
    function cancelListing(uint256 listingId) external nonReentrant listingExists(listingId) {
        Listing storage listing = listings[listingId];
        require(listing.seller == msg.sender, "Only seller can cancel");

        listing.active = false;
        emit ListingCanceled(listingId);
    }

    // --- Auction Functions ---

    /**
     * @dev Create an auction for an NFT.
     * @param tokenAddress The address of the NFT contract.
     * @param tokenId The ID of the NFT.
     * @param minBid The minimum starting bid.
     * @param bidIncrement The minimum increment over the current highest bid.
     * @param duration The auction duration in seconds.
     */
    function createAuction(
        address tokenAddress,
        uint256 tokenId,
        uint256 minBid,
        uint256 bidIncrement,
        uint256 duration
    ) external nonReentrant {
        require(minBid > 0, "Min bid must be >0");
        require(bidIncrement > 0, "Bid increment must be >0");
        IERC721 nft = IERC721(tokenAddress);
        require(nft.ownerOf(tokenId) == msg.sender, "You do not own this token");
        require(nft.isApprovedForAll(msg.sender, address(this)) || nft.getApproved(tokenId) == address(this),
            "Marketplace not approved to transfer token");

        _auctionIds.increment();
        uint256 auctionId = _auctionIds.current();

        auctions[auctionId] = Auction({
            id: auctionId,
            tokenAddress: tokenAddress,
            tokenId: tokenId,
            seller: payable(msg.sender),
            startTime: block.timestamp,
            endTime: block.timestamp + duration,
            minBid: minBid,
            bidIncrement: bidIncrement,
            highestBidder: payable(address(0)),
            highestBid: 0,
            ended: false
        });

        emit AuctionCreated(auctionId, tokenAddress, tokenId, msg.sender, block.timestamp, block.timestamp + duration, minBid);
    }

    /**
     * @dev Place a bid on an active auction.
     * @param auctionId The ID of the auction.
     */
    function placeBid(uint256 auctionId) external payable nonReentrant auctionExists(auctionId) auctionActive(auctionId) {
        Auction storage auction = auctions[auctionId];

        uint256 bidAmount = msg.value;
        require(bidAmount >= auction.minBid, "Bid below minimum");

        // If there's already a bid, check increment
        if (auction.highestBidder != address(0)) {
            require(bidAmount >= auction.highestBid + auction.bidIncrement, "Bid not high enough");
        }

        // Refund previous highest bidder
        if (auction.highestBidder != address(0)) {
            payable(auction.highestBidder).transfer(auction.highestBid);
        }

        // Update auction
        auction.highestBidder = payable(msg.sender);
        auction.highestBid = bidAmount;

        emit BidPlaced(auctionId, msg.sender, bidAmount);
    }

    /**
     * @dev End an auction and transfer NFT to winner.
     * @param auctionId The ID of the auction.
     */
    function endAuction(uint256 auctionId) external nonReentrant auctionExists(auctionId) {
        Auction storage auction = auctions[auctionId];
        require(block.timestamp >= auction.endTime, "Auction not ended yet");
        require(!auction.ended, "Auction already ended");

        auction.ended = true;

        if (auction.highestBidder == address(0)) {
            // No bids: return NFT to seller
            // (nothing to do, seller still owns it)
            emit AuctionEnded(auctionId, address(0), 0);
            return;
        }

        // Transfer payment to seller (after fees/royalties)
        _distributePayment(auction.tokenAddress, auction.tokenId, auction.seller, auction.highestBid);

        // Transfer NFT to winner
        IERC721(auction.tokenAddress).safeTransferFrom(auction.seller, auction.highestBidder, auction.tokenId);

        emit AuctionEnded(auctionId, auction.highestBidder, auction.highestBid);
    }

    // --- Internal Functions ---

    /**
     * @dev Distribute sale payment: platform fee, creator royalty, seller.
     */
    function _distributePayment(address tokenAddress, uint256 tokenId, address payable seller, uint256 amount) internal {
        uint256 remaining = amount;

        // Platform fee
        uint256 platformFee = (amount * platformFeeBasisPoints) / 10000;
        if (platformFee > 0) {
            feeRecipient.transfer(platformFee);
            remaining -= platformFee;
        }

        // Creator royalty (if contract supports EIP-2981)
        // We'll attempt to get royalty info
        (address receiver, uint256 royaltyAmount) = _getRoyaltyInfo(tokenAddress, tokenId, amount);
        if (receiver != address(0) && royaltyAmount > 0) {
            payable(receiver).transfer(royaltyAmount);
            remaining -= royaltyAmount;
        }

        // Seller gets the rest
        seller.transfer(remaining);
    }

    /**
     * @dev Try to get royalty info from the NFT contract (EIP-2981).
     */
    function _getRoyaltyInfo(address tokenAddress, uint256 tokenId, uint256 salePrice) internal view returns (address, uint256) {
        // Try to call royaltyInfo (EIP-2981)
        (bool success, bytes memory data) = tokenAddress.staticcall(
            abi.encodeWithSignature("royaltyInfo(uint256,uint256)", tokenId, salePrice)
        );
        if (success && data.length >= 64) {
            (address receiver, uint256 royaltyAmount) = abi.decode(data, (address, uint256));
            return (receiver, royaltyAmount);
        }
        return (address(0), 0);
    }

    // --- Admin Functions ---

    function setPlatformFee(uint256 _basisPoints) external onlyOwner {
        require(_basisPoints <= 1000, "Fee too high"); // max 10%
        platformFeeBasisPoints = _basisPoints;
    }

    function setFeeRecipient(address payable _feeRecipient) external onlyOwner {
        feeRecipient = _feeRecipient;
    }
}
```

**Explanation:**
- `listItem`: Creates a listing. Checks ownership and approval.
- `buyItem`: Handles purchase, distributes payment, transfers NFT, refunds excess.
- `createAuction`: Creates an auction, transferring no tokens yet.
- `placeBid`: Allows bidding, refunds previous bidder.
- `endAuction`: Ends auction, distributes funds, transfers NFT to winner.
- `_distributePayment`: Handles platform fee and royalty (EIP-2981 compatible).
- `_getRoyaltyInfo`: Attempts to call `royaltyInfo` on the NFT contract (standard for royalties).

### 36.2.3 Royalty System

Our marketplace supports EIP-2981 (NFT Royalty Standard). If the NFT contract implements `royaltyInfo`, we automatically pay royalties. In our `SimpleNFT`, we implemented a simple 5% royalty.

### 36.2.4 Auctions (Optional Enhancement)

The auction logic is a basic English auction. For production, you'd want to add features like:
- Minimum bid increment percentage.
- Reserve price (hidden).
- Anti-sniping extension (if bid near end, extend auction).
- Batch auctions.

But for learning, this suffices.

---

## 36.3 Frontend Development

We'll build a simple React frontend that interacts with the marketplace. It will have:

- **Home**: Display all active listings and auctions.
- **My NFTs**: Show NFTs owned by the user (requires indexing or user's wallet).
- **Create Listing**: Form to list an NFT.
- **Create Auction**: Form to start an auction.
- **NFT Detail**: Page for a specific NFT showing its listing/auction and buy/bid options.

We'll use ethers.js and IPFS for metadata.

### 36.3.1 NFT Display

We need to fetch NFT metadata from IPFS. Given a token contract and token ID, we call `tokenURI` to get the IPFS hash, then fetch the JSON from an IPFS gateway (e.g., `https://ipfs.io/ipfs/<hash>`).

**Helper:**
```javascript
export async function fetchNFTMetadata(tokenURI) {
  // tokenURI could be ipfs://Qm... or https://...
  const url = tokenURI.replace('ipfs://', 'https://ipfs.io/ipfs/');
  const response = await fetch(url);
  const metadata = await response.json();
  return metadata;
}
```

### 36.3.2 Listing Form

```javascript
import { useState } from 'react';
import { useWeb3 } from '../context/Web3Context';
import { getMarketplace, getNFTContract } from '../utils/contracts';
import { ethers } from 'ethers';

export default function ListNFT() {
  const { signer, account } = useWeb3();
  const [nftAddress, setNftAddress] = useState('');
  const [tokenId, setTokenId] = useState('');
  const [price, setPrice] = useState('');
  const [loading, setLoading] = useState(false);

  const handleList = async () => {
    if (!signer || !account) return;
    setLoading(true);
    try {
      const marketplace = getMarketplace(signer);
      const nft = getNFTContract(nftAddress, signer);

      // Check ownership
      const owner = await nft.ownerOf(tokenId);
      if (owner.toLowerCase() !== account.toLowerCase()) {
        alert('You do not own this NFT');
        return;
      }

      // Check approval
      const approved = await nft.getApproved(tokenId);
      const isApprovedForAll = await nft.isApprovedForAll(account, marketplace.address);
      if (approved.toLowerCase() !== marketplace.address.toLowerCase() && !isApprovedForAll) {
        // Need to approve
        const approveTx = await nft.approve(marketplace.address, tokenId);
        await approveTx.wait();
      }

      // List
      const priceWei = ethers.parseEther(price);
      const tx = await marketplace.listItem(nftAddress, tokenId, priceWei);
      await tx.wait();
      alert('NFT listed successfully');
    } catch (error) {
      console.error(error);
      alert('Failed to list NFT');
    } finally {
      setLoading(false);
    }
  };

  return (
    <div>
      <h2>List NFT for Sale</h2>
      <input placeholder="NFT Contract Address" value={nftAddress} onChange={e => setNftAddress(e.target.value)} />
      <input placeholder="Token ID" value={tokenId} onChange={e => setTokenId(e.target.value)} />
      <input placeholder="Price (ETH)" value={price} onChange={e => setPrice(e.target.value)} />
      <button onClick={handleList} disabled={loading}>List</button>
    </div>
  );
}
```

### 36.3.3 Buy Component

```javascript
import { useState } from 'react';
import { useWeb3 } from '../context/Web3Context';
import { getMarketplace } from '../utils/contracts';
import { ethers } from 'ethers';

export default function BuyNFT({ listing }) {
  const { signer } = useWeb3();
  const [loading, setLoading] = useState(false);

  const handleBuy = async () => {
    if (!signer) return;
    setLoading(true);
    try {
      const marketplace = getMarketplace(signer);
      const price = listing.price; // already in wei from contract
      const tx = await marketplace.buyItem(listing.id, { value: price });
      await tx.wait();
      alert('Purchase successful');
    } catch (error) {
      console.error(error);
      alert('Purchase failed');
    } finally {
      setLoading(false);
    }
  };

  return (
    <div>
      <p>Price: {ethers.formatEther(listing.price)} ETH</p>
      <button onClick={handleBuy} disabled={loading}>Buy Now</button>
    </div>
  );
}
```

### 36.3.4 Auction Components

Similar to listing, with `createAuction` and `placeBid` functions. We'll skip detailed code for brevity.

---

## 36.4 IPFS Integration

We need to upload NFT metadata (image + attributes) to IPFS. We'll use Pinata for easy uploading.

**Upload function:**
```javascript
import axios from 'axios';

export async function uploadToIPFS(file) {
  const formData = new FormData();
  formData.append('file', file);

  const response = await axios.post('https://api.pinata.cloud/pinning/pinFileToIPFS', formData, {
    headers: {
      'Content-Type': 'multipart/form-data',
      pinata_api_key: process.env.NEXT_PUBLIC_PINATA_API_KEY,
      pinata_secret_api_key: process.env.NEXT_PUBLIC_PINATA_SECRET_KEY
    }
  });
  return response.data.IpfsHash;
}

export async function uploadJSONToIPFS(json) {
  const response = await axios.post('https://api.pinata.cloud/pinning/pinJSONToIPFS', json, {
    headers: {
      'Content-Type': 'application/json',
      pinata_api_key: process.env.NEXT_PUBLIC_PINATA_API_KEY,
      pinata_secret_api_key: process.env.NEXT_PUBLIC_PINATA_SECRET_KEY
    }
  });
  return response.data.IpfsHash;
}
```

In the minting flow:
1. Upload image → get image CID.
2. Create metadata JSON with `image: ipfs://<imageCID>`.
3. Upload metadata → get metadata CID.
4. Call `mintNFT` with `tokenURI = ipfs://<metadataCID>`.

---

## 36.5 Testing and Security

### 36.5.1 Unit Tests

We need to test:
- Listing creation, purchase, cancellation.
- Auction creation, bidding, ending.
- Fee distribution and royalties.
- Access control (only seller can cancel, etc.).
- Edge cases (bidding after auction ended, insufficient payment).

**Hardhat test example (listing):**
```javascript
describe("Marketplace", function () {
  let marketplace, nft, owner, addr1, addr2;

  beforeEach(async function () {
    [owner, addr1, addr2] = await ethers.getSigners();
    const NFT = await ethers.getContractFactory("SimpleNFT");
    nft = await NFT.deploy();
    const Marketplace = await ethers.getContractFactory("NFTMarketplace");
    marketplace = await Marketplace.deploy(owner.address);
    // Mint an NFT to addr1
    await nft.mintNFT(addr1.address, "ipfs://test");
  });

  describe("Listings", function () {
    it("Should list an NFT", async function () {
      await nft.connect(addr1).approve(marketplace.address, 1);
      await marketplace.connect(addr1).listItem(nft.address, 1, ethers.parseEther("1"));
      const listing = await marketplace.listings(1);
      expect(listing.seller).to.equal(addr1.address);
      expect(listing.price).to.equal(ethers.parseEther("1"));
      expect(listing.active).to.be.true;
    });

    it("Should buy a listed NFT", async function () {
      await nft.connect(addr1).approve(marketplace.address, 1);
      await marketplace.connect(addr1).listItem(nft.address, 1, ethers.parseEther("1"));
      const sellerBalanceBefore = await ethers.provider.getBalance(addr1.address);
      const buyerBalanceBefore = await ethers.provider.getBalance(addr2.address);

      await marketplace.connect(addr2).buyItem(1, { value: ethers.parseEther("1") });

      const sellerBalanceAfter = await ethers.provider.getBalance(addr1.address);
      const buyerBalanceAfter = await ethers.provider.getBalance(addr2.address);
      const newOwner = await nft.ownerOf(1);
      expect(newOwner).to.equal(addr2.address);
      expect(sellerBalanceAfter).to.be.gt(sellerBalanceBefore);
      // Buyer spent 1 ETH + gas
    });
  });
});
```

### 36.5.2 Security Considerations

- **Reentrancy**: We use `nonReentrant` on all payable functions.
- **Approval checks**: Ensure marketplace is approved before listing.
- **Payment distribution**: Send funds after state changes (checks-effects-interactions).
- **Royalty enforcement**: Only call external contract for royalty info; if it fails, just skip.
- **Auction refunds**: Refund previous bidder before accepting new bid to avoid locked funds.
- **Access control**: Only seller can cancel listing/auction.

---

## 36.6 Complete Walkthrough

**User flow:**

1. **Creator mints NFT** using our `SimpleNFT` contract, uploading image and metadata to IPFS.
2. **Creator lists NFT** for sale at a fixed price via the marketplace.
3. **Buyer browses listings** (via subgraph or our contract events) and decides to buy.
4. **Buyer clicks "Buy"** and confirms transaction; NFT is transferred, payment distributed (fees, royalties, seller).
5. **Creator receives funds** minus platform fee and royalties.

For auctions:
1. Creator starts an auction.
2. Users place bids; each bid refunds previous highest bidder.
3. After auction ends, anyone can call `endAuction` to finalize.

---

## Chapter Summary

```
┌─────────────────────────────────────────────────────────────────┐
│                    CHAPTER 36 SUMMARY                           │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  We built a complete NFT marketplace:                          │
│    • NFT contract (ERC-721) with minting and royalties         │
│    • Marketplace contract with fixed-price listings and auctions│
│    • Support for EIP-2981 royalties and platform fees          │
│    • Frontend components for listing, buying, bidding          │
│    • IPFS integration for metadata storage                     │
│                                                                 │
│  Key features:                                                 │
│    • Listing and buying NFTs                                   │
│    • English auctions with bid increment and refunds           │
│    • Creator royalties on secondary sales                      │
│    • Platform fees                                             │
│                                                                 │
│  Security: reentrancy guard, approval checks, safe transfers  │
│  Testing: Hardhat tests cover core functionality               │
│                                                                 │
│  This project can be extended with features like:              │
│    • Batch listings                                            │
│    • Offer system                                              │
│    • Lazy minting                                              │
│    • Collection offers                                         │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Next Chapter Preview:** Chapter 37 – Building a DAO Governance System. We'll create a DAO with token-based voting, proposal creation, and execution via timelock, using OpenZeppelin Governor.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='35. building_a_decentralized_exchange.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='37. building_a_dao_governance_system.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
